In [1]:
import PyNormaliz
from PyNormaliz import *
import numpy
from numpy import *
import itertools
from itertools import *
from scipy.spatial import ConvexHull

In [2]:
import sys
sys.path.insert(0, './Class/')
sys.path.insert(0,'./ClassAffine')
import integerSmithNormalFormAndApplications
from integerSmithNormalFormAndApplications import *

Veamos algunos ejemplos de semigrupos afines y veamos si son $\mathcal{C}$-semigrupos.

In [3]:
gen0 = [[3,0],[4,0],[5,0],[2,2],[3,3],[4,3],[5,3],[5,1],[6,1],[7,1],[5,4],[5,5]]

In [4]:
gen1 = [[3,3],[4,4],[5,5],[5,0],[5,1],[5,2],[5,3],[5,4],[6,0]]

In [5]:
# En primer lugar calculamos el cono de los generadores.

In [6]:
C0 = Cone(cone=gen0)
C1 = Cone(cone=gen1)

In [7]:
rayos0 = C0.ExtremeRays()
rayos1 = C1.ExtremeRays()
rayos0,rayos1

([[1, 0], [1, 1]], [[1, 0], [1, 1]])

In [8]:
q0 = len(rayos0)
q1 = len(rayos1)
q0,q1

(2, 2)

Vemos ahora si en los ejes se forma un semigrupo.

In [9]:
# Check if all the coordinates are positives.
def BelongQPositive(v):
    for i in range(len(v)):
        if v[i] < 0:
            return False
    return True

In [10]:
# Pertenece a un eje y si es así, devuelve el múltiplo.
# INPUT:
#   - x: Value for checking if it is in the ray.
#   - r: Minimal value in the ray.
# OUTPUT:
#   - 0: If not belongs to the ray.
#   - A value if belongs to the ray.
def BelongAxis(x,r):
    coef = x[0]/r[0]
    aux2 = [j/coef for j in x]
    if aux2 == r:
        if(int(coef)==coef):
            return int(coef)
        return coef
    else:
        return 0

In [11]:
BelongAxis([3,0],[1,0])

3

In [12]:
# Vemos para un eje si se forma un semigrupo.
# INPUT:
#   - gen: Set of generators.
#   - r: Minimal value in the ray.
# OUTPUT:
#   - True/False.
def AxisIsSemigroup(gen,r):
    aux = []
    for x in gen:
        aux.append(BelongAxis(x,r))
    aux2 = [x for x in aux if x != 0]
    if(gcdL(aux2) == 1):
        return True
    else:
        return False

In [13]:
AxisIsSemigroup(gen0,[1,1])

True

In [14]:
# Vemos si los ejes forman un semigrupo.
# INPUT:
#   - gen: Set of generators.
#   - setR: Set of rays.
# OUTPUT:
#   - True/False.
def AxisAreSemigroup(gen,setR):
    aux = []
    for x in setR:
        aux.append(AxisIsSemigroup(gen,x))
    return all(aux)

In [15]:
AxisAreSemigroup(gen0,rayos0)

True

In [16]:
AxisAreSemigroup(gen1,rayos1)

True

Veamos ahora los puntos que hay dentro del *"diamantito"*

In [17]:
# En primer lugar tenemos que calcular el diamante.
# INPUT:
#   - a: Minimal elements of the ray.
# OUTPUT:
#   - Points of the diamond
def Diamond(a):
    aux = list(a)
    aux.append([0 for x in range(len(a))])
    for i in range(len(a)):
        for j in range(i+1,len(a)):
            aux.append(array(a[i])+array(a[j]))
    return [list(x) for x in  aux]

In [18]:
diamante0 = Diamond(rayos0)
diamante1 = Diamond(rayos1)
diamante0,diamante1

([[1, 0], [1, 1], [0, 0], [2, 1]], [[1, 0], [1, 1], [0, 0], [2, 1]])

In [19]:
'''
from scipy.spatial import ConvexHull
points = np.random.rand(30, 2)
hull = ConvexHull(points)
import matplotlib.pyplot as plt
plt.plot(points[:,0], points[:,1], 'o')
for simplex in hull.simplices:
     plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
plt.plot(points[hull.vertices,0], points[hull.vertices,1], 'r--', lw=2)
plt.plot(points[hull.vertices[0],0], points[hull.vertices[0],1], 'ro')
plt.show()
'''

"\nfrom scipy.spatial import ConvexHull\npoints = np.random.rand(30, 2)\nhull = ConvexHull(points)\nimport matplotlib.pyplot as plt\nplt.plot(points[:,0], points[:,1], 'o')\nfor simplex in hull.simplices:\n     plt.plot(points[simplex, 0], points[simplex, 1], 'k-')\nplt.plot(points[hull.vertices,0], points[hull.vertices,1], 'r--', lw=2)\nplt.plot(points[hull.vertices[0],0], points[hull.vertices[0],1], 'ro')\nplt.show()\n"

In [20]:
hull0 = ConvexHull(diamante0)
eq0 = [list(x) for x in hull0.equations]
hull1 = ConvexHull(diamante1)
eq1 = [list(x) for x in hull1.equations]
eq0,eq1

([[-0.7071067811865475, 0.7071067811865475, -0.0], [-0.0, 1.0, -1.0], [-0.0, -
1.0, 0.0], [0.7071067811865475, -0.7071067811865475, -0.7071067811865475]], [[
-0.7071067811865475, 0.7071067811865475, -0.0], [-0.0, 1.0, -1.0], [-0.0, -1.0
, 0.0], [0.7071067811865475, -0.7071067811865475, -0.7071067811865475]])

In [21]:
# Ahora veremos si un punto pertenece al diamante o no.
# INPUT:
#   - pt: Punto para comprobar la pertenencia.
#   - eq: Ecuaciones del diamante.
# OUTPUT:
#   - True/False.
def PointBelongsDiamond(pt,eq):
    dim = len(pt)
    for x in eq:
        sum = 0
        for i in range(dim):
            sum = sum + pt[i]*x[i]
        sum = round(sum+x[-1],10)
        if sum > 0:
            return False
    return True

In [22]:
PointBelongsDiamond([1,1],eq0)

True

In [23]:
# Calculo el menor cuboide que contenga al diamante.
# INPUT:
#   - d: Vértices del diamante.
# OUTPUT:
#   - Máximos en cada coordenada del cubo.
def Cube(d):
    dim = len(d[0])
    aux = []
    for i in range(dim):
        aux.append(sorted(d, key = lambda x: x[i])[-1][i])
    return(aux)

In [24]:
cube0 = Cube(diamante0)
cube1 = Cube(diamante1)
cube0,cube1

([2, 1], [2, 1])

In [25]:
# Calculamos todos los puntos del diamante.
# INPUT:
#   - eq: Ecuaciones que definen el diamante.
# OUTPUT:
#   - Puntos enteros del diamante.
def IntegerDiamond(eq,cube):
    d = []
    it = product(*[range(i+1) for i in cube])
    for x in it:
        if PointBelongsDiamond(list(x),eq):
            d.append(list(x))
    return d

In [26]:
diamanteEntero0 = IntegerDiamond(eq0,cube0)
diamanteEntero1 = IntegerDiamond(eq1,cube1)
diamanteEntero0,diamanteEntero1

([[0, 0], [1, 0], [1, 1], [2, 1]], [[0, 0], [1, 0], [1, 1], [2, 1]])

In [27]:
eq=ConvexHull(Diamond([[1,3],[5,2]])).equations
IntegerDiamond(eq,[6,5])

[[0, 0], [1, 1], [1, 2], [1, 3], [2, 1], [2, 2], [2, 3], [3, 2], [3, 3], [4, 2
], [4, 3], [4, 4], [5, 2], [5, 3], [5, 4], [6, 5]]

A continuación, veremos para cada rayo el semigrupo afín generado por los términos independientes de las ecuaciones en los puntos enteros del diamante.

In [28]:
'''C0.SupportHyperplanes()'''

'C0.SupportHyperplanes()'

In [29]:
'''Cone(cone=[[1,0,0],[3,4,5],[4,5,5]]).SupportHyperplanes()'''

'Cone(cone=[[1,0,0],[3,4,5],[4,5,5]]).SupportHyperplanes()'

In [30]:
def ProdEsc(v1,v2):
    n = len(v1)
    suma = 0
    for i in range(n):
        suma = suma + v1[i] * v2[i]
    return suma

In [31]:
ProdEsc([1,2,3],[1,1,1])

6

In [37]:
# En primer lugar calculamos las ecuaciones que definen un rayo.
# INPUT:
#   - ray: rayo del cono.
#   - x: valor interno del cono para calibrar las ecuaciones.
# OUTPUT:
#   - ecuaciones del rayo.
def EqRay(ray,x):
    n = len(ray)
    eq = []
    for j in range(1,n):
        aux = []
        aux = aux + [-ray[j]]                 # Valor -aj
        aux = aux + [0 for k in range(j-1)]   # Ceros intermediso
        aux = aux + [ray[0]]                  # Vaor ai
        aux = aux + [0 for k in range(n-j-1)] # Ceros finales
        eq.append(aux)
    # Calibramos las ecuaciones
    for i in range(len(eq)):
        calibre = ProdEsc(eq[i],x)
        if calibre < 0:
            for j in range(len(eq[i])):
                eq[i][j] = -eq[i][j]
    return eq

In [38]:
EqRay([1,2,3,4],[1,1,1,1])

[[2, -1, 0, 0], [3, 0, -1, 0], [4, 0, 0, -1]]

In [39]:
EqRay(rayos0[0],diamanteEntero0[0])

[[0, 1]]

In [30]:
# Eliminamos los elementos que son todos ceros.
# INPUT:
#   - m: matriz.
# OUTPUT:
#   - Matriz m sin filas nulas.
def DeleteRowZero(m):
    aux = []
    for v in m:
        allzero = True
        for i in range(len(v)):
            if v[i] != 0:
                allzero = False
        if not allzero:
            aux.append(v)
    return aux

In [35]:
DeleteRowZero([[0,1],[0,0],[1,0],[1]])

[[0, 1], [1, 0], [1]]

In [32]:
# Calculamos los valores afines del diamante con respecto a un rayo.
# INPUT:
#   - eq: Ecuaciones del rayo.
#   - d: Diamante entero.
# OUTPUT:
#   - Terminos afines del diamante.
def AffineTerm(eq,d):
    neq = len(eq)
    dim = len(eq[0])
    afin = []
    for x in d:
        aux = []
        for e in eq:
            aux = aux + [ProdEsc(x,e)]
        afin.append(aux)
    return afin

In [33]:
AffineTerm([[0,1]],diamanteEntero0)

NameError: name 'diamanteEntero0' is not defined

In [43]:
rayos0[0]

[1, 0]

In [44]:
diamanteEntero0

[[0, 0], [1, 0], [1, 1], [2, 1]]

In [ ]:
# Nos quedamos con los términos